In [25]:
#Este es el enfoque recomendado y más común para desarrollar aplicaciones.
#Claridad y Control: Importas y usas la clase ChatGoogleGenerativeAI directamente. Esto hace que tu código sea más claro y te da control total sobre todos los parámetros del modelo (como temperature, top_p, etc.) desde el inicio.
#Manejo de API Key: Asume que la clave ya está disponible en el entorno. No incluye código para solicitarla.
#Uso Ideal: Para construir cualquier aplicación o script robusto, ya que es la forma más transparente y flexible de trabajar.

In [26]:
import langchain
import os

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [28]:
try:
    with open('../gem_apikey.txt') as f:
        api_key = f.read().strip()
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'. Asegúrate de que la ruta es correcta.")
    api_key = None

In [29]:

def main():
    # 1. Configuración de la clave de API de Google
    load_dotenv()
    if not api_key:
        raise ValueError("La variable de entorno GOOGLE_API_KEY no está configurada.")

    # 2. Carga del documento PDF del profesor
    # Asegúrate de tener un archivo 'ejemplo.pdf' en el mismo directorio.
    print("Cargando el documento PDF...")
    try:
        loader = PyPDFLoader("../Docs/Doc1.pdf")
        documents = loader.load()
        print(f"Documento cargado. Número de páginas: {len(documents)}")
    except Exception as e:
        print(f"Error al cargar el PDF: {e}")
        return

    # 3. División del texto en fragmentos
    print("Dividiendo el texto en fragmentos...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    print(f"Número de fragmentos: {len(texts)}")

    # 4. Creación de Embeddings con Gemini y la base de datos vectorial (FAISS)
    print("Creando embeddings y la base de datos vectorial (FAISS) con Gemini...")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vectorstore = FAISS.from_documents(texts, embeddings)
    print("Base de datos vectorial creada.")

    # 5. Configuración del modelo de lenguaje (LLM) de Gemini
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, google_api_key=api_key)

    # 6. Creación de la cadena de Retrieval-Augmented Generation (RAG)
    print("Configurando la cadena de RAG...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever()
    )
    print("Cadena de RAG lista. ¡El asistente de profesor está listo!")

    # 7. Bucle de preguntas y respuestas
    print("\n¡El asistente de profesor está listo para responder preguntas sobre el PDF!")
    print("Escribe 'salir' para terminar la conversación.")
    while True:
        query = input("\nPregunta del estudiante: ")
        if query.lower() == "salir":
            break
        
        # Generación de la respuesta
        try:
            result = qa_chain.invoke({"query": query})
            print("\nRespuesta del asistente:")
            print(result["result"])
        except Exception as e:
            print(f"Ocurrió un error: {e}")

if __name__ == "__main__":
    main()

Cargando el documento PDF...
Documento cargado. Número de páginas: 11
Dividiendo el texto en fragmentos...
Número de fragmentos: 33
Creando embeddings y la base de datos vectorial (FAISS) con Gemini...
Base de datos vectorial creada.
Configurando la cadena de RAG...
Cadena de RAG lista. ¡El asistente de profesor está listo!

¡El asistente de profesor está listo para responder preguntas sobre el PDF!
Escribe 'salir' para terminar la conversación.



Pregunta del estudiante:  saludame



Respuesta del asistente:
No tengo información para saludarte.



Pregunta del estudiante:  cuanto es 2+2



Respuesta del asistente:
No tengo información sobre eso.


...


Pregunta del estudiante:  describe la estructura de los vasos sanguineos



Respuesta del asistente:
Los vasos sanguíneos forman una red de conductos que transportan la sangre desde el corazón a los tejidos y desde los tejidos al corazón.

Su estructura general y clasificación es la siguiente:

*   **Arterias:** Son vasos que distribuyen la sangre del corazón a los tejidos. A medida que se ramifican, su calibre disminuye progresivamente.
*   **Arteriolas:** Son las ramificaciones más pequeñas de las arterias.
*   **Capilares:** En el interior de los tejidos, las arteriolas se ramifican en múltiples vasos microscópicos que se distribuyen entre las células.
*   **Vénulas:** Los capilares se unen en grupos formando venas pequeñas llamadas vénulas.
*   **Venas:** Las vénulas se fusionan para dar lugar a venas de mayor calibre, que son las encargadas de retornar la sangre al corazón.

En cuanto a la estructura de sus paredes, el texto indica que:

*   Las paredes de los **grandes vasos (arterias y venas)** están constituidas por **tres capas**. Sin embargo, el tex


Pregunta del estudiante:  puedes generar un diagrama mermeid que represente esta estructura?



Respuesta del asistente:
```mermaid
graph TD
    A[Corazón]

    subgraph Estructura y Composición
        A -- "Formado por" --> B[Miocardio (Músculo Cardíaco)]
        B -- "Fibras" --> B1[Fibras Musculares Estriadas]
        B1 -- "Conectadas por" --> B2[Discos Intercalares]
        B -- "Incluye" --> B3[Fibras Autorrítmicas (1%)]
    end

    subgraph Sistema de Conducción Cardíaco
        A -- "Posee" --> C[Sistema de Conducción Cardíaco]
        C -- "Componente clave" --> C1[Nódulo Sinusal]
        C -- "Componente clave" --> C2[Nódulo Auriculoventricular (Nódulo AV)]
        C1 -- "Genera" --> D[Potencial de Acción]
    end

    subgraph Fisiología del Potencial de Acción
        D -- "Causa" --> E[Contracción del Músculo Cardíaco]
        D -- "Fases" --> D1[Despolarización]
        D -- "Fases" --> D2[Meseta]
        D -- "Fases" --> D3[Repolarización]
        D -- "Se propaga desde" --> C1
        D -- "Se propaga por" --> F[Miocardio Auricular]
        F -- "Hasta" --> C2



Pregunta del estudiante:  salir
